In [50]:
#libraries
import pandas as pd


# Extract Data

In [51]:

PATH = r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes\data.xlsx'
data = pd.read_excel(PATH)
data.head(1)

,Marital Status,AGE,Gender,Car Value,Years of No Claims Bonus,Annual Mileage,Payment Method,Acquisition Channel,Years of Tenure with Current Provider,Price,Actual Change in Price vs last Year,% Change in Price vs last Year,Grouped Change in Price,Renewed?
0,M,45.0,F,500.0,4.0,6000.0,Monthly,Inbound,4.0,289.4,-11.94,-0.0396,-0.05,0.0


# Transform Data

## Remove Unwanted Columns

In [52]:
data.drop(columns=['Marital Status','Car Value','Grouped Change in Price', 'Gender'], inplace=True)

## Policyholder age.

### EDA

In [53]:
#rename column
data.rename(columns={"AGE":"age"}, inplace= True)
#check descriptive statistics
data.age.describe()

count    20017.000000
mean        44.743718
std         12.680774
min         17.000000
25%         36.000000
50%         44.000000
75%         52.000000
max         89.000000
Name: age, dtype: float64

In [54]:
#check discretization
print(pd.qcut(data.age, q = 20))
#apply
data['age_discrete'] = pd.qcut(data.age, q = 20)
#verifiy
data['age_discrete'].value_counts()
data['age'] = data.age.astype(int)

0          (44.0, 45.0]
1          (39.0, 41.0]
2        (16.999, 25.0]
3          (41.0, 42.0]
4          (55.0, 59.0]
              ...      
20012      (47.0, 49.0]
20013      (45.0, 47.0]
20014      (49.0, 50.0]
20015      (49.0, 50.0]
20016      (47.0, 49.0]
Name: age, Length: 20017, dtype: category
Categories (20, interval[float64, right]): [(16.999, 25.0] < (25.0, 29.0] < (29.0, 31.0] < (31.0, 33.0] ... (55.0, 59.0] < (59.0, 63.0] < (63.0, 68.0] < (68.0, 89.0]]


## Price , Actual Change in Price vs Last Year -- Expiring and Renewing Premium + Policy Renewed or Lapsed.


In [55]:
#rename columns according to game
data.rename(columns={"Price":"renewing_premium","% Change in Price vs last Year":"%_chg_rnw_price_vs_actual", "Renewed?":'renewed'}, inplace =True)
#calculate expiring premium
data['expiring_premium'] = data['renewing_premium'] - data['Actual Change in Price vs last Year']
#Change Renewed value from 0 to -1, to calculate the non aggregated version of net revenue "net_revenue"
data['renewed_2'] = data['renewed']
data.loc[data['renewed'] == 0,'renewed_2'] = -1
data['renewed'].value_counts() 
#creating net revenue metric
data['net_revenue'] = data['renewed_2'] * data['renewing_premium']
#remove Actual Change in Price vs last Year	(not needed anymore)
data.drop(columns=['Actual Change in Price vs last Year'], inplace = True)
data.head(1)


,age,Years of No Claims Bonus,Annual Mileage,Payment Method,Acquisition Channel,Years of Tenure with Current Provider,renewing_premium,%_chg_rnw_price_vs_actual,renewed,age_discrete,expiring_premium,renewed_2,net_revenue
0,45,4.0,6000.0,Monthly,Inbound,4.0,289.4,-0.0396,0.0,"(44.0, 45.0]",301.34,-1.0,-289.4


## Acquisition Column

In [56]:
data['Acquisition Channel'].value_counts()
#remove outbound category
data = data[data['Acquisition Channel'] != 'Outbound']
data['Acquisition Channel'].value_counts()

Inbound    16064
Direct      3938
Aggreg        12
Name: Acquisition Channel, dtype: int64

# Data Load

In [57]:
data.to_excel(r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes\processed.xlsx', index = False) # to tableau

# Monthly Profiling

## Data Extract

In [12]:
# Choose an UK indicator
## https://www.ons.gov.uk/economy/nationalaccounts/uksectoraccounts/timeseries/npyh/ukea
## ESA:LEVEL:ASSET:FinCos: Insurance Premiums & Reserves
## Quarterly estimates of national product, income and expenditure, sector accounts and balance of payments.

#So first step get data from a file that is online and txt format and convert it into a table
##Learned something: With Python 3 urllib.urlretrieve() is considered legacy. no need for inventing headers anymore :)))
import pandas as pd
import requests
PATH = r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes'
url ='https://www.ons.gov.uk/generator?format=csv&uri=/economy/nationalaccounts/uksectoraccounts/timeseries/npyh/ukea'
r = requests.get(url)
with open (rf"{PATH}\historical_data.txt", 'wb') as outfile:
    outfile.write(r.content)

historical_indicator_data = pd.read_csv(rf"{PATH}\historical_data.txt")
historical_indicator_data

,Title,ESA:LEVEL:ASSET:FinCos: Insurance Premiums & Reserves
0,CDID,NPYH
1,Source dataset ID,UKEA
2,PreUnit,NaN
3,Unit,NaN
4,Release date,31-03-2022
...,...,...
177,2020 Q4,573672
178,2021 Q1,393198
179,2021 Q2,349782
180,2021 Q3,348107


# Data Processing

In [36]:
import numpy as np
#Only Need the Quaterly Profiling So everything that on title has a "Q" is a good start
historical_data_ind_subset = historical_indicator_data[historical_indicator_data.Title.str.contains("Q") ]
#rename column titles
historical_data_ind_subset = historical_data_ind_subset.rename(columns={'Title':'QTR Date','ESA:LEVEL:ASSET:FinCos: Insurance Premiums & Reserves':'Ind_Value'})
#transform Ind_Value to numeric
historical_data_ind_subset.Ind_Value = pd.to_numeric(historical_data_ind_subset.Ind_Value)
# now get this table as profile to recreate the trend on ourt data
##get last value first which will be supposed to be the last point and coincides to the last point on the data I got  (assumption)
last_value = historical_data_ind_subset.Ind_Value.values[-1]
last_value
#Divide everything by last value
historical_data_ind_subset['profile'] = historical_data_ind_subset['Ind_Value']/ last_value

,QTR Date,Ind_Value,profile
42,1987 Q1,129099,0.364586
43,1987 Q2,130052,0.367277
44,1987 Q3,131001,0.369957
45,1987 Q4,131950,0.372637
46,1988 Q1,126410,0.356992
...,...,...,...
177,2020 Q4,573672,1.620094
178,2021 Q1,393198,1.110421
179,2021 Q2,349782,0.987811
180,2021 Q3,348107,0.983081


In [ ]:
historical_data_ind_subset.to_csv(rf'{PATH}\Profile.csv')

# SQL

In [1]:
import sqlite3
import pandas as pd
#creating sql instance
conn= sqlite3.connect("mock_database.db")

In [2]:
#creating table 
data= pd.read_excel(r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes\processed.xlsx', index_col = None)
data.to_sql('dbo.insurance', conn, if_exists='replace')

20014

In [7]:
PATH = r'C:\Users\ppuen\Documents\Entrevista Insurance Lunes'
historical_data_ind_subset = pd.read_csv(rf'{PATH}\Profile.csv')
historical_data_ind_subset.to_sql('dbo.profile',conn,if_exists='replace')

140

In [3]:
#testing
cursor = conn.cursor()
cursor.execute("""SELECT * FROM 'dbo.insurance' LIMIT 5""")
cursor.fetchone()

(0,
 45,
 4,
 6000,
 'Monthly',
 'Inbound',
 4,
 289.4,
 -0.0396,
 0,
 '(44.0, 45.0]',
 301.34,
 -1,
 -289.4)

In [5]:
#now with pandas
data_sql = pd.read_sql("""SELECT * FROM 'dbo.insurance'""", conn)
data_sql.drop(columns = ['index'],inplace = True)
data_sql

,age,Years of No Claims Bonus,Annual Mileage,Payment Method,Acquisition Channel,Years of Tenure with Current Provider,renewing_premium,%_chg_rnw_price_vs_actual,renewed,age_discrete,expiring_premium,renewed_2,net_revenue
0,45,4,6000,Monthly,Inbound,4,289.4,-0.0396,0,"(44.0, 45.0]",301.34,-1,-289.4
1,40,8,6000,Monthly,Inbound,4,170.4,0.3700,1,"(39.0, 41.0]",124.78,1,170.4
2,25,4,4000,Monthly,Inbound,4,466.1,-0.2100,1,"(16.999, 25.0]",589.25,1,466.1
3,42,9,10000,Annual,Inbound,4,245.1,0.0100,1,"(41.0, 42.0]",242.76,1,245.1
4,59,9,3000,Annual,Inbound,4,240.5,0.2200,0,"(55.0, 59.0]",197.94,-1,-240.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20009,47,8,10000,Annual,Aggreg,3,412.9,-0.2300,0,"(45.0, 47.0]",533.09,-1,-412.9
20010,50,8,30000,Annual,Aggreg,3,610.5,0.5400,1,"(49.0, 50.0]",396.27,1,610.5
20011,50,7,6999,Annual,Aggreg,3,210.6,0.1200,0,"(49.0, 50.0]",187.70,-1,-210.6
20012,50,7,7999,Annual,Aggreg,3,263.4,0.2700,0,"(49.0, 50.0]",208.17,-1,-263.4


In [10]:
data_historic = pd.read_sql("""SELECT * FROM 'dbo.profile'""", conn)
data_historic.drop(columns = ['Unnamed: 0','index'],inplace = True)
data_historic

,QTR Date,Ind_Value,profile
0,1987 Q1,129099,0.364586
1,1987 Q2,130052,0.367277
2,1987 Q3,131001,0.369957
3,1987 Q4,131950,0.372637
4,1988 Q1,126410,0.356992
...,...,...,...
135,2020 Q4,573672,1.620094
136,2021 Q1,393198,1.110421
137,2021 Q2,349782,0.987811
138,2021 Q3,348107,0.983081


In [46]:
#Here I am going to suposse 2021 Q4 matches expiring premium net revenue
#And 2022 Q1 will be the value from renewing premium net revenue * 3
query = """
WITH BASE AS (
        SELECT      age_discrete
                ,   expiring_premium AS '2021 Q4'
                ,   CASE WHEN renewed == 0 THEN 0 ELSE renewing_premium END AS [2022 Q1]
                ,   [Acquisition Channel]
                ,   [Years of Tenure with Current Provider]
        FROM    'dbo.insurance'
        )
,
BASE2 AS (
        SELECT      age_discrete
                ,   [Acquisition Channel]
                ,   [Years of Tenure with Current Provider]
                ,   Sum([2021 Q4]) * 3 AS [2021 Q4]
                ,   Sum([2022 Q1]) * 3 AS [2022 Q1]
        FROM BASE
        GROUP BY age_discrete,[Acquisition Channel],[Years of Tenure with Current Provider]
        )
        
SELECT * FROM BASE2
"""


In [49]:
processed_data = pd.read_sql(query, conn)
processed_data.head(4)


,age_discrete,Acquisition Channel,Years of Tenure with Current Provider,2021 Q4,2022 Q1
0,"(16.999, 25.0]",Direct,1,36481.38,7659.3
1,"(16.999, 25.0]",Direct,2,192129.93,92500.2
2,"(16.999, 25.0]",Direct,3,136230.90,67466.4
3,"(16.999, 25.0]",Direct,4,55905.96,21120.3


In [60]:
processed_data.columns

Index(['age_discrete', 'Acquisition Channel',
       'Years of Tenure with Current Provider', '2021 Q4', '2022 Q1'],
      dtype='object')

In [70]:
#Now we need to apply the profiling to this values
#in order to get the historical values for each category
##List of Quarters except last one
quarters_list = list(data_historic['QTR Date'].value_counts().index)[:-1]
quarters_list.sort()
quarters_list #good

#Remove last 2 columns from processed data
pre_processed_data = processed_data[['age_discrete', 'Acquisition Channel', 'Years of Tenure with Current Provider']]
last_2_columns = processed_data[['2021 Q4', '2022 Q1']]
#Create Empty Columns and Restore Configuration keeping order
df = pd.DataFrame()
df = df.reindex(columns=quarters_list)
pre_processed_data = pd.concat([pre_processed_data,df,last_2_columns],axis=1)
pre_processed_data.head(5)



,age_discrete,Acquisition Channel,Years of Tenure with Current Provider,1987 Q1,1987 Q2,1987 Q3,1987 Q4,1988 Q1,1988 Q2,1988 Q3,...,2019 Q4,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1
0,"(16.999, 25.0]",Direct,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36481.38,7659.3
1,"(16.999, 25.0]",Direct,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,192129.93,92500.2
2,"(16.999, 25.0]",Direct,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136230.90,67466.4
3,"(16.999, 25.0]",Direct,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55905.96,21120.3
4,"(16.999, 25.0]",Inbound,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149891.94,54893.4


In [84]:
#Populate Data Using the Profiling Obtained
for i in range(len(data_historic)):
    sub_set = data_historic.iloc[i,:]
    pre_processed_data[f'{data_historic.iloc[i,0]}'] = data_historic.iloc[i,2] * pre_processed_data['2021 Q4']


In [89]:
pre_processed_data

,age_discrete,Acquisition Channel,Years of Tenure with Current Provider,1987 Q1,1987 Q2,1987 Q3,1987 Q4,1988 Q1,1988 Q2,1988 Q3,...,2019 Q4,2020 Q1,2020 Q2,2020 Q3,2020 Q4,2021 Q1,2021 Q2,2021 Q3,2021 Q4,2022 Q1
0,"(16.999, 25.0]",Direct,1,13300.582541,13398.766533,13496.538420,13594.310307,13023.545024,12454.325134,11884.899192,...,45453.727829,56710.785209,62125.637510,62575.656058,59103.260192,40509.705373,36036.718816,35864.149890,36481.38,7659.3
1,"(16.999, 25.0]",Direct,2,70047.788559,70564.876549,71079.794181,71594.711813,68588.764837,65590.956707,62592.063398,...,239382.982386,298668.504110,327185.933921,329555.965760,311268.522282,213345.187536,189788.112825,188879.275067,192129.93,92500.2
2,"(16.999, 25.0]",Direct,3,49667.812185,50034.456582,50399.562073,50764.667564,48633.282507,46507.668347,44381.284735,...,169735.965319,211772.726491,231993.184224,233673.669771,220706.846310,151273.707895,134570.420233,133926.003271,136230.90,67466.4
3,"(16.999, 25.0]",Direct,4,20382.502951,20532.965196,20682.795909,20832.626623,19957.956282,19085.654182,18213.036317,...,69655.651454,86906.550396,95204.551078,95894.182856,90572.903222,62079.174861,55224.538124,54960.084546,55905.96,21120.3
4,"(16.999, 25.0]",Inbound,1,54648.429424,55051.840397,55453.558145,55855.275893,53510.158587,51171.390877,48831.776555,...,186756.845395,233008.992914,255257.129257,257106.131493,242839.013504,166443.219177,148064.949695,147355.911520,149891.94,54893.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,"(68.0, 89.0]",Direct,4,2132.136168,2147.875452,2163.548673,2179.221895,2087.725955,1996.477747,1905.196509,...,7286.412966,9090.963941,9958.986255,10031.126014,9474.487176,6493.866545,5776.829047,5749.165564,5848.11,4349.1
159,"(68.0, 89.0]",Inbound,1,28341.233284,28550.446333,28758.781257,28967.116181,27750.914410,26538.005605,25324.657737,...,96854.006211,120840.842000,132378.952589,133337.864021,125938.791011,86319.152317,76787.994180,76420.279746,77735.49,28722.6
160,"(68.0, 89.0]",Inbound,2,151498.590516,152616.942763,153730.600982,154844.259201,148343.029978,141859.403361,135373.429730,...,517734.894580,645956.972164,707634.157273,712760.036198,673208.153575,461420.636826,410471.653443,408506.029084,415536.51,264798.9
161,"(68.0, 89.0]",Inbound,3,51503.791897,51883.989371,52262.591053,52641.192735,50431.020641,48226.832768,46021.846999,...,176010.286153,219600.943849,240568.854480,242311.459481,228865.315033,156865.568025,139544.840297,138876.602344,141266.70,80756.4


In [94]:
pre_processed_data
pre_processed_unpivoted = pre_processed_data.melt(id_vars=['age_discrete', 'Acquisition Channel',
       'Years of Tenure with Current Provider'], var_name='QTR Date', value_name='Quaterly Value')
pre_processed_unpivoted


,age_discrete,Acquisition Channel,Years of Tenure with Current Provider,QTR Date,Quaterly Value
0,"(16.999, 25.0]",Direct,1,1987 Q1,13300.582541
1,"(16.999, 25.0]",Direct,2,1987 Q1,70047.788559
2,"(16.999, 25.0]",Direct,3,1987 Q1,49667.812185
3,"(16.999, 25.0]",Direct,4,1987 Q1,20382.502951
4,"(16.999, 25.0]",Inbound,1,1987 Q1,54648.429424
...,...,...,...,...,...
22978,"(68.0, 89.0]",Direct,4,2022 Q1,4349.100000
22979,"(68.0, 89.0]",Inbound,1,2022 Q1,28722.600000
22980,"(68.0, 89.0]",Inbound,2,2022 Q1,264798.900000
22981,"(68.0, 89.0]",Inbound,3,2022 Q1,80756.400000


In [95]:
#THIS ONE GOES TO TABLEAU
pre_processed_unpivoted.to_csv(rf'{PATH}\Processed_Profile.csv')